In [1]:
#import needed libraries
import itertools
import pandas as pd
import json
import warnings
warnings.simplefilter('ignore')

In [2]:
#load the json file
with open('roomlist.json') as f:
   data = json.load(f)

In [3]:
data=data['value'][0]

In [4]:
#Compress the address to summarize the json file
data['Address']=data['Address']['StreetAddress']+','+data['Address']['City']+' '+data['Address']['PostalCode']+','+data['Address']['Country']

In [5]:
#delete the french description , last rennovation data
del data['Description_fr']
del data['LastRenovationDate']

In [6]:
#Create DataFrame
df = pd.DataFrame(columns=['Type', 'BaseRate', 'BedOptions','SleepsCount','SmokingAllowed','Tags','Description'])

In [7]:
#delete the french description in rooms and convert the json file data for rooms to df
for room in data['Rooms']:
    del room['Description_fr']
    df.loc[len(df)] = [room['Type'], room['BaseRate'], room['BedOptions'],room['SleepsCount'],room['SmokingAllowed'],room['Tags'],room['Description']]


In [8]:
#sort depending on the baserate
df=df.sort_values('BaseRate',ascending=False)
df=df.reset_index(drop=True)

In [9]:
#Encoding the Smoking Allowed Values
df['SmokingAllowed'] = df['SmokingAllowed'].map({True: 1, False: 0})

In [10]:
#remove the duplicated values in each list of tags for each row
for i in range(len(df['Tags'])):
   df['Tags'][i] = list(set( df['Tags'][i]))

In [11]:
# define a function to extract view from the description
def extract_view(s):
    start_index = s.index('(') + 1  # move the start index one character to the right to skip the opening parenthesis
    end_index = s.index(')')  # find the index of the closing parenthesis
    return s[start_index:end_index]  # extract the text between the parentheses

# apply the function to the 'Room_Type' column
df['Description'] = df['Description'].apply(extract_view)
df.rename(columns={'Description': 'View'}, inplace=True)


In [12]:
#descriptive statistics on the room level in the data
print(df['BaseRate'].describe())

count     13.000000
mean     159.528462
std       74.393789
min       55.990000
25%       96.990000
50%      146.990000
75%      234.990000
max      262.990000
Name: BaseRate, dtype: float64


In [13]:
#Mean value for each room type with each view
df.groupby(['Type','View'])['BaseRate'].mean()

Type           View           
Budget Room    Cityside            96.99
               Mountain View       80.99
               Waterfront View     87.99
Deluxe Room    City View          148.99
Standard Room  Amenities          110.99
               City View           55.99
               Waterfront View    121.99
Suite          Mountain View      239.49
               Waterfront View    262.99
Name: BaseRate, dtype: float64

In [14]:
#explode tags to show the effect of each single tag on the baserate
df_exploded= df.copy()
df_exploded=df_exploded.explode('Tags')

In [15]:
#the effect of each single tag on the baserate
df_exploded.groupby(['Tags'])['BaseRate'].mean().sort_values()

Tags
jacuzzi tub         68.49
suite              103.49
bathroom shower    132.74
coffee maker       151.99
vcr/dvd            158.59
tv                 161.49
Room Tags          177.49
Name: BaseRate, dtype: float64

In [16]:
#the effect of each single tag and room type on the baserate
df_exploded.groupby(['Tags','Type'])['BaseRate'].mean()

Tags             Type         
Room Tags        Standard Room    110.99
                 Suite            243.99
bathroom shower  Deluxe Room      148.99
                 Standard Room    116.49
coffee maker     Deluxe Room      150.99
                 Standard Room     55.99
                 Suite            248.99
jacuzzi tub      Budget Room       80.99
                 Standard Room     55.99
suite            Deluxe Room      150.99
                 Standard Room     55.99
tv               Budget Room       87.99
                 Suite            234.99
vcr/dvd          Budget Room       88.99
                 Standard Room    121.99
                 Suite            246.49
Name: BaseRate, dtype: float64

In [17]:
#max base rate for each room type
df.groupby(['Type'])['BaseRate'].max().sort_values()

Type
Budget Room       96.99
Standard Room    121.99
Deluxe Room      150.99
Suite            262.99
Name: BaseRate, dtype: float64

In [18]:
#min base rate for each room type
df.groupby(['Type'])['BaseRate'].min().sort_values()

Type
Standard Room     55.99
Budget Room       80.99
Deluxe Room      146.99
Suite            229.99
Name: BaseRate, dtype: float64

In [19]:
#mean base rate for each room type
df.groupby(['Type'])['BaseRate'].mean().sort_values()

Type
Budget Room       88.656667
Standard Room     96.323333
Deluxe Room      148.990000
Suite            244.190000
Name: BaseRate, dtype: float64

In [20]:
#max base rate for each bed option
df.groupby(['BedOptions'])['BaseRate'].max().sort_values()

BedOptions
2 Queen Beds     229.99
1 Queen Bed      243.99
2 Double Beds    248.99
1 King Bed       262.99
Name: BaseRate, dtype: float64

In [21]:
#min base rate for each bed option
df.groupby(['BedOptions'])['BaseRate'].min().sort_values()

BedOptions
2 Double Beds     55.99
1 King Bed        80.99
1 Queen Bed       96.99
2 Queen Beds     229.99
Name: BaseRate, dtype: float64

In [22]:
#mean base rate for each bed option

df.groupby(['BedOptions'])['BaseRate'].mean().sort_values()

BedOptions
1 King Bed       149.561429
2 Double Beds    151.990000
1 Queen Bed      170.490000
2 Queen Beds     229.990000
Name: BaseRate, dtype: float64

In [23]:
#mean base rate for each smoking option(1: smoking allowed,0:smoking not allowed)
df.groupby(['SmokingAllowed'])['BaseRate'].mean().sort_values()

SmokingAllowed
1    136.823333
0    178.990000
Name: BaseRate, dtype: float64

In [24]:
#Create all possible combinations for reserving 6 people

# create a list of the room indices
room_indices = list(df.index)

# create a list of all possible combinations of 3 room indices for 6 people
room_combinations = []
for combination in itertools.combinations(room_indices, 3):
    if sum(df.loc[list(combination)]['SleepsCount']) == 6:
        room_combinations.append(combination)
for combination in itertools.combinations(room_indices, 2):
    if sum(df.loc[list(combination)]['SleepsCount']) == 6:
        room_combinations.append(combination)

In [25]:
#Select the combinations that have at least one smoking allowed room
smoking=[]
for x in room_combinations:
    for i in (x) :
        if (df['SmokingAllowed'][i]==1):
            s=1
            break;
        else :
            s=0
    smoking.append(s)

In [26]:
#Select the combinations that have at least one City view room
view=[]
for x in room_combinations:
    for i in (x) :
        if (df['View'][i]=='City View'):
            v=1
            break;
        else :
            v=0
    view.append(v)

In [27]:
#Select the combinations that have at least one Suite Type room

suite=[]
for x in room_combinations:
    for i in (x) :
        if (df['Type'][i]=='Suite'):
            r=1
            break;
        else :
            r=0
    suite.append(r)

In [28]:
#Make and operation to select the combinations that have the smoking and city view conditions
smoking_view = [a and b for a, b in zip(smoking, view)]

In [29]:
#Make and operation with suite condition to select the combinations that apply all the three conditions
smoking_view_suite = [a and b for a, b in zip(smoking_view, suite)]

In [30]:
#final_combinations list is the list that contain the constrained combinations
final_combinations=[]
for i in range(len(room_combinations)):
    if smoking_view_suite[i] == 1:
        final_combinations.append(room_combinations[i])

In [31]:
#calculate the total cost for the 10 nights on the constrained combinations
total_cost=[]
for x in final_combinations:
    cost=0
    for i in x:
        cost=cost+(df['BaseRate'][i])
    cost=cost*10
    total_cost.append(cost)

In [32]:
#select the minimum cost and its index
cheapest_cost=min(total_cost)
cheapest_index = total_cost.index(cheapest_cost)

In [33]:
#Create a list for the cheapest combination
cheapest_list=[]
for i in final_combinations[cheapest_index]:
    cheapest_list.append([df['Type'][i],df['SleepsCount'][i],df['SmokingAllowed'][i],df['View'][i],df['BaseRate'][i]])

In [34]:
#Convert it to df to be easy to understand
cheapest_df=pd.DataFrame(cheapest_list, columns=['Type','People','SmokingAllowed','View','Base Rate'])
cheapest_df

,Type,People,SmokingAllowed,View,Base Rate
0,Suite,4,1,Mountain View,229.99
1,Standard Room,2,1,City View,55.99


In [35]:
#Print the details of the cheapest combination
print('The Cheapest Cost for 6 people in 10 nights is',cheapest_cost)
print('-----------------------------Details-----------------------------')
print(cheapest_df)

The Cheapest Cost for 6 people in 10 nights is 2859.8
-----------------------------Details-----------------------------
            Type  People  SmokingAllowed           View  Base Rate
0          Suite       4               1  Mountain View     229.99
1  Standard Room       2               1      City View      55.99
